In [2]:
from nifti_utils.NiftiHandlerCustom import NiftiHandlerDjangoS3 as nh
from nifti_utils.AtlasUtils import CustomAtlasLabeler
from nifti_utils.AnalysisUtils import ImageComparison, GroupAnalysis

# s3_path1 = "uploads/lesion_files/AliceWonderlandLesionNetwork_Hong_2010_Case01.nii.gz"
# s3_path2 = "uploads/lesion_files/AliceWonderlandLesionNetwork_Kim_2006_Case01.nii.gz"
local_path = r"C:\Users\josep\a_repos\Takotsubo\PALM_data\all_vs_symptoms_vox_tstat.nii"

img = nh.load(local_path)
df = img.normalize_to_quantile().apply_anatomical_mask().convert_to_dataframe().df_xyz
# Sort by value
df.sort_values(by='value', ascending=False, inplace=True)

,x,y,z,value
122999,-4.0,-40.0,-60.0,1.000
106587,4.0,-38.0,-52.0,1.000
106652,4.0,-36.0,-54.0,1.000
106518,4.0,-40.0,-60.0,1.000
119003,-2.0,-34.0,-48.0,1.000
...,...,...,...,...
118735,-2.0,-44.0,72.0,0.001
118797,-2.0,-42.0,72.0,0.001
118861,-2.0,-40.0,72.0,0.001
190036,-36.0,-62.0,56.0,0.001


In [10]:
emotion_regulation1 = r"C:\Users\josep\a_repos\Takotsubo\emotion_regulation_data\Figure_3C.nii"
emotion_regulation2 = r"C:\Users\josep\a_repos\Takotsubo\emotion_regulation_data\Figure_4A.nii"
medulla_func_t = r"C:\Users\josep\a_repos\Takotsubo\emotion_regulation_data\precom_t_medulla_seed.nii.gz"

data = [
    {'path':emotion_regulation1, 'symptom':'emotion_regulation1'},
    {'path':emotion_regulation2, 'symptom':'emotion_regulation2'},
    {'path':medulla_func_t, 'symptom':'medulla_func_t'}
]

df1, df2 = GroupAnalysis(data).cross_correlation_analysis()

display(df1)


c:\Users\josep\miniconda3\envs\ni_env\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\josep\miniconda3\envs\ni_env\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


,correlation,path1,path2,symptom1,symptom2
0,-0.212197,C:\Users\josep\a_repos\Takotsubo\emotion_regul...,C:\Users\josep\a_repos\Takotsubo\emotion_regul...,emotion_regulation1,emotion_regulation2
1,-0.134790,C:\Users\josep\a_repos\Takotsubo\emotion_regul...,C:\Users\josep\a_repos\Takotsubo\emotion_regul...,emotion_regulation1,medulla_func_t
2,0.033760,C:\Users\josep\a_repos\Takotsubo\emotion_regul...,C:\Users\josep\a_repos\Takotsubo\emotion_regul...,emotion_regulation2,medulla_func_t
